In [0]:
import csv
import time
from datetime import datetime

**Check the validity of the light switch**

In [0]:
def checktransition(state1, state2):
 
  state1 = int(state1) 
  state2 = int(state2)
  
  if (state1  == state2-1):
    return True
  if  (state1==3 and state2==0):
    return True
  return False 

**Check time**: validity and duration

In [0]:
def checktime(time1, time2, curr_state):
  diff= time2-time1
  
  #adjust the limits according to the traffic light
  minforYellow=1
  minforRG=10
  maxforYellow=3.5
  maxforRG=120
  
  if (diff <= 0):
    print("wrong sequence: time1>time2: " , time1, time2)
    return False
  
  #case Yellow
  if(curr_state == '1' or curr_state == '3'):
    #print("Yellow Duration: " , diff)
    if(diff < minforYellow or diff > maxforYellow):
      print("yellow duration should be between: " , minforYellow , maxforYellow )
      return False
    return True
  
  if(curr_state == '0' or curr_state == '2' ):  
    if (diff < minforRG or diff > maxforRG):
      print("duration is too short or too long: " , diff , time1, time2 , )
      return False
  return True

**Filtering the files** 

---

adjust filename and copy


In [0]:
filename= 'enterFileName.csv'
with open(filename, 'r+') as f:
    lines = f.readlines()
copy = open('fileName_filterd.csv', "w+") 
copy.write(lines[0])
i =0
y=1
counter = 0
addcounter = False
while i < len(lines) and y < len(lines):
  line = lines[i]
  list1 = line.split(",")
  curr_state = list1[0]
  curr_time = list1[2].split(":")[0]

        
  nextline = lines[y]
  nextlist1 = nextline.split(",")
  next_state = nextlist1[0]
  next_time = nextlist1[2].split(":")[0]

      
  if (checktransition(curr_state,next_state)):
    if(checktime(int(curr_time), int(next_time) , curr_state)):
      copy.write(lines[y])
    else:
      print("can't continue, duration error. Line:" , i+1 )
      break;
    if addcounter :
      i= i + counter + 1
      counter = 0
    else:
      i+=1
    addcounter = False
    y+=1
  elif (int(curr_state)==3 and int(next_state)==2 ):
    print("deleted line:" , y+1)
    counter+=1
    y+=1
    addcounter = True
  else:
    addcounter = False
    #other wrong transition which should be checked manualy 
    print("wrong transition: " , curr_state , next_state , i)
    print("i: " , i , "y: " , y)
    i+=1
    y+=1

copy.close()


deleted line: 739
deleted line: 752
wrong transition:  3 3 750
i:  750 y:  752
wrong transition:  2 2 751
i:  751 y:  753


**Pushing the results**


In [0]:
import requests
from requests.auth import HTTPBasicAuth
import time
from datetime import datetime
from getpass import getpass

In [0]:
def push_results (status_id, time, milisec,username,password):
  
  ts = int(time)

  #extract time
  day = datetime.utcfromtimestamp(ts).strftime('%A')
  hour = datetime.utcfromtimestamp(ts).strftime('%H')
  minute = datetime.utcfromtimestamp(ts).strftime('%M')
  seconds = datetime.utcfromtimestamp(ts).strftime('%S')
  millis = milisec

  #status:
  if(status_id == '0'):
    status='green'
  elif (status_id == '2'):
    status = 'red'
  else:
    print("status can only be green or red")
    return -1
  

  print (status , day , hour , minute , seconds, millis)


  #build request check the region and intersection
  url = "https://werkzeug.dcaiti.tu-berlin.de/0432l770/trafficlights/timeset?region=copy&intersection=20021&signalGroup=1&status={}&day={}&hour={}&minute={}&second={}&millis={}".format(status,day,hour,minute,seconds,millis)

  request = requests.get(url,auth=HTTPBasicAuth(username,password), verify=True)
  if(request.ok):
      print("request has been sent successfully")
  else:
      request.raise_for_status()
  return 1


In [0]:
#go through the copied file and call the function push_results on all the lines where status is 0 or 2
filename='filename_filterd.csv'
i=0
#get auth
username = input("Please enter your username: ")
password = getpass("Please enter password: ")

with open(filename, 'r+') as f:
    lines = f.readlines()
while i < len(lines):
  line = lines[i]
  list1 = line.split(",")
  state = list1[0]
  if (state == '0' or state == '2' ):
    time = list1[2].split(":")[0]
    milisec = list1[2].split(":")[1]
    push_results(state, time, milisec, username, password)
    i+=1
  else:
    i+=1
  
  

Please enter your username: reemalrashid
Please enter password: ··········
red Sunday 05 20 42 29

request has been sent successfully
green Sunday 05 20 54 05

request has been sent successfully
red Sunday 05 21 26 09

request has been sent successfully
green Sunday 05 21 54 05

request has been sent successfully
red Sunday 05 22 26 09

request has been sent successfully
green Sunday 05 22 54 05

request has been sent successfully
red Sunday 05 23 26 09

request has been sent successfully
green Sunday 05 23 54 05

request has been sent successfully
red Sunday 05 24 26 09

request has been sent successfully
green Sunday 05 24 54 05

request has been sent successfully
red Sunday 05 25 26 09

request has been sent successfully
green Sunday 05 25 54 05

request has been sent successfully
red Sunday 05 26 26 09

request has been sent successfully
green Sunday 05 26 54 05

request has been sent successfully
red Sunday 05 27 26 09

request has been sent successfully
green Sunday 05 27 54 05

